In [1]:
#!uv add google-auth-oauthlib google-auth-httplib2 google-api-python-client

Resolved 240 packages in 5.92s                                       
Prepared 10 packages in 353ms                                            
░░░░░░░░░░░░░░░░░░░░ [0/10] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 10 packages in 155msent==2.185.0                  
 + google-api-core==2.28.1
 + google-api-python-client==2.185.0
 + google-auth-httplib2==0.2.0
 + google-auth-oauthlib==1.2.2
 + httplib2==0.31.0
 + oauthlib==3.3.1
 + proto-plus==1.26.1
 + pyparsing==3.2.5
 + requests-oauthlib==2.0.0
 + uritemplate==4.2.0


In [2]:
#!uv add PyPDF2 python-docx openpyxl

Resolved 245 packages in 251ms                                       
Prepared 5 packages in 88ms                                              
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 52ms                                
 + et-xmlfile==2.0.0
 + lxml==6.0.2
 + openpyxl==3.1.5
 + pypdf2==3.0.1
 + python-docx==1.2.0


In [3]:
#!uv add pandas 

Resolved 245 packages in 110ms                                       
Audited 237 packages in 1ms


# Gmail Connection

In [12]:
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly',
          'https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    """Authenticate and return Gmail API service"""
    creds = None
    
   
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # If no valid credentials, let user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # save credentials for next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return build('gmail', 'v1', credentials=creds)


service = authenticate_gmail()

# Elasticsearch

In [13]:
# structuring data before storing it in Elasticsearch

from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from datetime import datetime
import base64


@dataclass
class EmailAttachment:
    """Represents an email attachment"""
    filename: str
    mime_type: str
    attachment_id: str
    size: int
    content: Optional[bytes] = None
    
    def to_es_dict(self) -> Dict:
        """Convert to Elasticsearch-friendly format"""
        return {
            'filename': self.filename,
            'mime_type': self.mime_type,
            'size': self.size,
            'has_content': self.content is not None
        }


@dataclass
class EmailDocument:
    """Elasticsearch-optimized email document structure"""
    
    # Required fields first 
    email_id: str
    thread_id: str
    sender_name: str
    sender_email: str
    subject: str
    
    # Optional fields with defaults
    recipients: List[str] = field(default_factory=list)
    cc: List[str] = field(default_factory=list)
    bcc: List[str] = field(default_factory=list)
    body_plain: str = ""
    body_html: str = ""
    snippet: str = ""
    date: datetime = field(default_factory=datetime.now)
    processed_at: Optional[datetime] = None
    attachments: List[EmailAttachment] = field(default_factory=list)
    labels: List[str] = field(default_factory=list)
    category: Optional[str] = None
    priority: str = "normal"
    is_read: bool = False
    is_important: bool = False
    is_starred: bool = False
    has_attachments: bool = False
    extracted_data: Dict[str, Any] = field(default_factory=dict)
    embedding: Optional[List[float]] = None
    action_items: List[Dict[str, Any]] = field(default_factory=list)
    processing_status: str = "pending"
    processing_errors: List[str] = field(default_factory=list)
    
    def to_es_document(self) -> Dict:
        """Convert to Elasticsearch document format"""
        return {
            '_id': self.email_id,
            '_source': {
                'email_id': self.email_id,
                'thread_id': self.thread_id,
                'sender': {
                    'name': self.sender_name,
                    'email': self.sender_email
                },
                'recipients': self.recipients,
                'cc': self.cc,
                'bcc': self.bcc,
                'subject': self.subject,
                'body': {
                    'plain': self.body_plain,
                    'html': self.body_html
                },
                'snippet': self.snippet,
                'date': self.date.isoformat(),
                'processed_at': self.processed_at.isoformat() if self.processed_at else None,
                'attachments': [att.to_es_dict() for att in self.attachments],
                'has_attachments': len(self.attachments) > 0,
                'attachment_count': len(self.attachments),
                'labels': self.labels,
                'category': self.category,
                'priority': self.priority,
                'is_read': self.is_read,
                'is_important': self.is_important,
                'is_starred': self.is_starred,
                'extracted_data': self.extracted_data,
                'action_items': self.action_items,
                'action_count': len(self.action_items),
                'processing_status': self.processing_status,
                'processing_errors': self.processing_errors,
                'embedding': self.embedding
            }
        }
    
    def get_context_for_llm(self, include_body: bool = True, max_body_length: int = 1000) -> str:
        """Get formatted context for LLM"""
        body = self.body_plain[:max_body_length] if include_body else self.snippet
        if len(self.body_plain) > max_body_length and include_body:
            body += "\n[... truncated ...]"
        
        return f"""
Subject: {self.subject}
From: {self.sender_name} <{self.sender_email}>
Date: {self.date.strftime('%Y-%m-%d %H:%M')}
Body: {body}
Attachments: {len(self.attachments)} file(s)
        """.strip()


class ElasticsearchEmailStore:
    """Manages email documents in Elasticsearch"""
    
    def __init__(self, es_host: str = "localhost", es_port: int = 9200, 
                 index_name: str = "emails"):
        self.es = Elasticsearch([f"http://{es_host}:{es_port}"])
        self.index_name = index_name
        
        # Query ES to check if index exists
        if not self.es.indices.exists(index=self.index_name):
            print(f"Index '{self.index_name}' doesn't exist. Creating...")
            self.create_index()
        else:
            print(f"Index '{self.index_name}' already exists")
    
    def create_index(self):
        """Create Elasticsearch index with optimized mapping"""
        mapping = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 1,
                "analysis": {
                    "analyzer": {
                        "email_analyzer": {
                            "type": "custom",
                            "tokenizer": "standard",
                            "filter": ["lowercase", "stop", "snowball"]
                        }
                    }
                }
            },
            "mappings": {
                "properties": {
                    "email_id": {"type": "keyword"},
                    "thread_id": {"type": "keyword"},
                    "sender": {
                        "properties": {
                            "name": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
                            "email": {"type": "keyword"}
                        }
                    },
                    "recipients": {"type": "keyword"},
                    "cc": {"type": "keyword"},
                    "bcc": {"type": "keyword"},
                    "subject": {
                        "type": "text",
                        "analyzer": "email_analyzer",
                        "fields": {"keyword": {"type": "keyword"}}
                    },
                    "body": {
                        "properties": {
                            "plain": {"type": "text", "analyzer": "email_analyzer"},
                            "html": {"type": "text", "index": False}
                        }
                    },
                    "snippet": {"type": "text"},
                    "date": {"type": "date"},
                    "processed_at": {"type": "date"},
                    "attachments": {
                        "type": "nested",
                        "properties": {
                            "filename": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
                            "mime_type": {"type": "keyword"},
                            "size": {"type": "long"},
                            "has_content": {"type": "boolean"}
                        }
                    },
                    "has_attachments": {"type": "boolean"},
                    "attachment_count": {"type": "integer"},
                    "labels": {"type": "keyword"},
                    "category": {"type": "keyword"},
                    "priority": {"type": "keyword"},
                    "is_read": {"type": "boolean"},
                    "is_important": {"type": "boolean"},
                    "is_starred": {"type": "boolean"},
                    "extracted_data": {"type": "object", "enabled": True},
                    "action_items": {
                        "type": "nested",
                        "properties": {
                            "description": {"type": "text"},
                            "priority": {"type": "keyword"},
                            "due_date": {"type": "date"},
                            "type": {"type": "keyword"},
                            "completed": {"type": "boolean"}
                        }
                    },
                    "action_count": {"type": "integer"},
                    "processing_status": {"type": "keyword"},
                    "processing_errors": {"type": "text"},
                    "embedding": {
                        "type": "dense_vector",
                        "dims": 1536,
                        "index": True,
                        "similarity": "cosine"
                    }
                }
            }
        }
        
        self.es.indices.create(index=self.index_name, body=mapping)
        print(f"Created index: {self.index_name}")
    
    def search(self, query: Dict) -> List[Dict]:
        """Execute a search query"""
        result = self.es.search(index=self.index_name, body=query)
        return [hit['_source'] for hit in result['hits']['hits']]

print("classes loaded!")

classes loaded!


Ran this in terminal: 

docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    -v es9_data:/usr/share/elasticsearch/data \
    docker.elastic.co/elasticsearch/elasticsearch:9.1.1
    

In [10]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '1576000f09ac', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'X2EE6hfCRtyf9yI4QAjjnw', 'version': {'number': '9.1.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '5e94055934defa56e454868b7783b2a3b683785e', 'build_date': '2025-08-05T01:07:31.959947279Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
# Initialize ElasticSearchEmailStore

es_store = ElasticsearchEmailStore(
    es_host="localhost",
    es_port=9200,
    index_name="emails",
)

print(f"Index name: {es_store.index_name}")

✅ Index 'emails' already exists
Index name: emails


# Email Fetching